In [1]:
# Supabase to PostgreSQL Migration
# 競馬データをSupabaseからローカルPostgreSQLに移行

import os
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
from supabase import create_client, Client
from dotenv import load_dotenv
import json
from datetime import datetime

# 環境変数読み込み
load_dotenv()


True

In [2]:
# =============================================================================
# 1. 接続設定
# =============================================================================

print("🔌 データベース接続を設定中...")

# Supabase接続
supabase_url = os.getenv('NEXT_PUBLIC_SUPABASE_URL')
supabase_key = os.getenv('SUPABASE_SERVICE_ROLE_KEY')
supabase: Client = create_client(supabase_url, supabase_key)

# PostgreSQL接続設定
pg_config = {
    'host': os.getenv('POSTGRES_HOST', 'localhost'),
    'database': os.getenv('POSTGRES_DB', 'stallion_db'),
    'user': os.getenv('POSTGRES_USER', 'stallion_user'),
    'password': os.getenv('POSTGRES_PASSWORD'),
    'port': int(os.getenv('POSTGRES_PORT', '5432'))
}

print("✅ 接続設定完了")

🔌 データベース接続を設定中...
✅ 接続設定完了


In [3]:
# =============================================================================
# 2. Supabaseからデータ取得
# =============================================================================

def fetch_supabase_horses():
    """Supabaseからhorsesデータを全件取得"""
    print("📥 Supabaseからhorsesデータを取得中...")
    
    try:
        # 全件取得（大量データの場合はページネーション必要）
        response = supabase.table('horses').select("*").execute()
        
        if response.data:
            print(f"✅ {len(response.data)}件のhorsesデータを取得")
            return response.data
        else:
            print("⚠️ Supabaseにhorsesデータが存在しません")
            return []
            
    except Exception as e:
        print(f"❌ Supabaseデータ取得エラー: {e}")
        return []

# データ取得実行
supabase_horses = fetch_supabase_horses()

# データ確認
if supabase_horses:
    print(f"\n📊 取得したデータの最初の3件:")
    for i, horse in enumerate(supabase_horses[:3]):
        print(f"  {i+1}. ID: {horse.get('id')}, Name: {horse.get('name_ja')}")

📥 Supabaseからhorsesデータを取得中...
✅ 692件のhorsesデータを取得

📊 取得したデータの最初の3件:
  1. ID: 2019105213, Name: ナミュール
  2. ID: 2022105185, Name: エンブロイダリー
  3. ID: 2022103795, Name: ナチュラルライズ


In [4]:
# =============================================================================
# 3. データ変換・クリーニング
# =============================================================================

def clean_horse_data(horses_data):
    """データをPostgreSQL用に変換・クリーニング"""
    print("\n🧹 データクリーニング中...")
    
    cleaned_data = []
    
    for horse in horses_data:
        # 基本的なクリーニング
        cleaned_horse = {
            'id': horse.get('id'),
            'name_ja': horse.get('name_ja'),
            'name_en': horse.get('name_en'),
            'birth_date': horse.get('birth_date'),
            'sex': horse.get('sex'),
            'profile': json.dumps(horse.get('profile')) if horse.get('profile') else None,
            'sire_id': horse.get('sire_id'),
            'dam_id': horse.get('dam_id'),
            'maternal_grandsire_id': horse.get('maternal_grandsire_id'),
            'created_at': horse.get('created_at'),
            'updated_at': horse.get('updated_at')
        }
        
        # None値や空文字の処理
        for key, value in cleaned_horse.items():
            if value == '' or value == 'null':
                cleaned_horse[key] = None
        
        cleaned_data.append(cleaned_horse)
    
    print(f"✅ {len(cleaned_data)}件のデータをクリーニング完了")
    return cleaned_data

# データクリーニング実行
if supabase_horses:
    cleaned_horses = clean_horse_data(supabase_horses)
else:
    cleaned_horses = []


🧹 データクリーニング中...
✅ 692件のデータをクリーニング完了


In [5]:
# データ確認
if cleaned_horses:
    print(f"\n📊 取得したデータの最初の3件:")
    for i, horse in enumerate(cleaned_horses[:3]):
        print(horse)


📊 取得したデータの最初の3件:
{'id': '2019105213', 'name_ja': 'ナミュール', 'name_en': 'Namur', 'birth_date': '2019-03-02', 'sex': 'mare', 'profile': '{"owner": "\\u30ad\\u30e3\\u30ed\\u30c3\\u30c8\\u30d5\\u30a1\\u30fc\\u30e0", "breeder": "\\u30ce\\u30fc\\u30b6\\u30f3\\u30d5\\u30a1\\u30fc\\u30e0", "trainer": "\\u9ad8\\u91ce\\u53cb\\u548c(\\u6817\\u6771)", "birthplace": "\\u5b89\\u5e73\\u753a", "career_record": {"wins": 5, "first": 5, "third": 2, "others": 7, "second": 4, "starts": 18, "win_rate": 27.8}, "offering_info": {"raw_text": "1\\u53e3:6.5\\u4e07\\u5186/400\\u53e3"}, "related_horses": "\\u30e9\\u30f4\\u30a7\\u30eb\\u3001\\u30f4\\u30a7\\u30b9\\u30bf\\u30fc\\u30f4\\u30a1\\u30eb\\u30c8", "total_prize_central": 50963}', 'sire_id': '000a011996', 'dam_id': '2010104050', 'maternal_grandsire_id': '2001103114', 'created_at': '2025-08-23T07:09:57.679334', 'updated_at': '2025-08-23T08:50:10.980668'}
{'id': '2022105185', 'name_ja': 'エンブロイダリー', 'name_en': 'Embroidery', 'birth_date': '2022-02-01', 'sex': 'mar

In [6]:
# =============================================================================
# 4. PostgreSQLへの接続テスト
# =============================================================================

def test_postgresql_connection():
    """PostgreSQL接続テスト（詳細版）"""
    print("🔗 PostgreSQL接続テスト中...")
    
    try:
        conn = psycopg2.connect(**pg_config)
        cursor = conn.cursor()
        
        # 現在接続しているデータベース確認
        cursor.execute("SELECT current_database();")
        current_db = cursor.fetchone()[0]
        print(f"接続中のデータベース: {current_db}")
        
        # 現在のユーザー確認
        cursor.execute("SELECT current_user;")
        current_user = cursor.fetchone()[0]
        print(f"接続中のユーザー: {current_user}")
        
        # 全テーブル一覧確認
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
        tables = cursor.fetchall()
        print(f"publicスキーマ内のテーブル一覧: {[t[0] for t in tables]}")
        
        # horsesテーブル存在確認
        cursor.execute("SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'horses');")
        table_exists = cursor.fetchone()[0]
        
        if table_exists:
            print("✅ horsesテーブルが存在します")
            
            # テーブル詳細確認
            cursor.execute("SELECT COUNT(*) FROM horses;")
            count = cursor.fetchone()[0]
            print(f"horsesテーブルの行数: {count}")
        else:
            print("❌ horsesテーブルが存在しません")
        
        cursor.close()
        conn.close()
        return table_exists
        
    except Exception as e:
        print(f"❌ PostgreSQL接続エラー: {e}")
        return False

# 接続テスト実行
pg_ready = test_postgresql_connection()

🔗 PostgreSQL接続テスト中...
接続中のデータベース: stallion_db
接続中のユーザー: keiba_user
publicスキーマ内のテーブル一覧: ['horses']
✅ horsesテーブルが存在します
horsesテーブルの行数: 3


In [8]:
# =============================================================================
# 5. データ移行実行
# =============================================================================

def migrate_horses_to_postgresql(horses_data):
    """horsesデータをPostgreSQLに移行"""
    print(f"\n📤 {len(horses_data)}件のhorsesデータをPostgreSQLに移行中...")
    
    if not horses_data:
        print("移行するデータがありません")
        return False
    
    try:
        conn = psycopg2.connect(**pg_config)
        cursor = conn.cursor()
        
        # 既存データ削除（必要に応じて）
        # cursor.execute("TRUNCATE horses CASCADE;")
        
        success_count = 0
        error_count = 0
        
        for horse in horses_data:
            try:
                # UPSERT（存在すれば更新、なければ挿入）
                upsert_query = """
                INSERT INTO horses (id, name_ja, name_en, birth_date, sex, profile, 
                                  sire_id, dam_id, maternal_grandsire_id, created_at, updated_at)
                VALUES (%(id)s, %(name_ja)s, %(name_en)s, %(birth_date)s, %(sex)s, %(profile)s,
                       %(sire_id)s, %(dam_id)s, %(maternal_grandsire_id)s, %(created_at)s, %(updated_at)s)
                ON CONFLICT (id) DO UPDATE SET
                    name_ja = EXCLUDED.name_ja,
                    name_en = EXCLUDED.name_en,
                    birth_date = EXCLUDED.birth_date,
                    sex = EXCLUDED.sex,
                    profile = EXCLUDED.profile,
                    sire_id = EXCLUDED.sire_id,
                    dam_id = EXCLUDED.dam_id,
                    maternal_grandsire_id = EXCLUDED.maternal_grandsire_id,
                    updated_at = EXCLUDED.updated_at;
                """
                
                cursor.execute(upsert_query, horse)
                success_count += 1
                
                if success_count % 100 == 0:
                    print(f"  {success_count}件処理完了...")
                
            except Exception as e:
                error_count += 1
                print(f"  ⚠️ データ挿入エラー (ID: {horse.get('id')}): {e}")
        
        conn.commit()
        cursor.close()
        conn.close()
        
        print(f"\n✅ 移行完了: {success_count}件成功, {error_count}件エラー")
        return True
        
    except Exception as e:
        print(f"❌ 移行エラー: {e}")
        return False

# 移行実行
if cleaned_horses and pg_ready:
    migration_success = migrate_horses_to_postgresql(cleaned_horses)
else:
    print("移行条件が満たされていません")
    migration_success = False



📤 692件のhorsesデータをPostgreSQLに移行中...
  100件処理完了...
  200件処理完了...
  300件処理完了...
  400件処理完了...
  500件処理完了...
  600件処理完了...

✅ 移行完了: 692件成功, 0件エラー


In [10]:
# =============================================================================
# horse_relations移行用の関数を追加
# =============================================================================

def fetch_supabase_horse_relations():
    """Supabaseからhorse_relationsデータを全件取得（ページネーション対応）"""
    print("📥 Supabaseからhorse_relationsデータを全件取得中...")
    
    all_data = []
    page_size = 1000
    offset = 0
    
    try:
        while True:
            response = supabase.table('horse_relations').select("*").range(offset, offset + page_size - 1).execute()
            
            if not response.data:
                break
            
            all_data.extend(response.data)
            print(f"  {len(all_data)}件取得完了...")
            
            # 取得件数がpage_sizeより少ない場合は最後のページ
            if len(response.data) < page_size:
                break
            
            offset += page_size
        
        print(f"✅ 合計{len(all_data)}件のhorse_relationsデータを取得")
        return all_data
        
    except Exception as e:
        print(f"❌ Supabaseデータ取得エラー: {e}")
        return all_data  # 部分的に取得できたデータを返す

def clean_horse_relations_data(relations_data):
    """horse_relationsデータをPostgreSQL用に変換"""
    print(f"\n🧹 horse_relationsデータクリーニング中...")
    
    cleaned_data = []
    
    for relation in relations_data:
        cleaned_relation = {
            'horse_a_id': relation.get('horse_a_id'),
            'horse_b_id': relation.get('horse_b_id'),
            'relation_type': relation.get('relation_type'),
            'children_ids': relation.get('children_ids'),  # 配列はそのまま
            'created_at': relation.get('created_at')
        }
        
        # None値処理
        for key, value in cleaned_relation.items():
            if value == '' or value == 'null':
                cleaned_relation[key] = None
        
        cleaned_data.append(cleaned_relation)
    
    print(f"✅ {len(cleaned_data)}件のrelationsデータをクリーニング完了")
    return cleaned_data

def migrate_horse_relations_to_postgresql(relations_data):
    """horse_relationsデータをPostgreSQLに移行"""
    print(f"\n📤 {len(relations_data)}件のhorse_relationsデータをPostgreSQLに移行中...")
    
    if not relations_data:
        print("移行するデータがありません")
        return False
    
    try:
        conn = psycopg2.connect(**pg_config)
        cursor = conn.cursor()
        
        success_count = 0
        error_count = 0
        
        for relation in relations_data:
            try:
                # UPSERT（idは自動採番のため、重複チェックは他の条件で）
                upsert_query = """
                INSERT INTO horse_relations (horse_a_id, horse_b_id, relation_type, children_ids, created_at)
                VALUES (%(horse_a_id)s, %(horse_b_id)s, %(relation_type)s, %(children_ids)s, %(created_at)s)
                ON CONFLICT DO NOTHING;
                """
                
                cursor.execute(upsert_query, relation)
                success_count += 1
                
                if success_count % 100 == 0:
                    print(f"  {success_count}件処理完了...")
                
            except Exception as e:
                error_count += 1
                print(f"  ⚠️ データ挿入エラー: {e}")
        
        conn.commit()
        cursor.close()
        conn.close()
        
        print(f"\n✅ 移行完了: {success_count}件成功, {error_count}件エラー")
        return True
        
    except Exception as e:
        print(f"❌ 移行エラー: {e}")
        return False

# horse_relations移行実行
supabase_relations = fetch_supabase_horse_relations()

if supabase_relations:
    cleaned_relations = clean_horse_relations_data(supabase_relations)
    relations_migration_success = migrate_horse_relations_to_postgresql(cleaned_relations)
else:
    relations_migration_success = False

# 結果確認
if relations_migration_success:
    try:
        conn = psycopg2.connect(**pg_config)
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM horse_relations;")
        count = cursor.fetchone()[0]
        print(f"📊 PostgreSQLのhorse_relationsテーブル: {count}件")
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"確認エラー: {e}")

📥 Supabaseからhorse_relationsデータを取得中...
✅ 1000件のhorse_relationsデータを取得

🧹 horse_relationsデータクリーニング中...
✅ 1000件のrelationsデータをクリーニング完了

📤 1000件のhorse_relationsデータをPostgreSQLに移行中...
  100件処理完了...
  200件処理完了...
  300件処理完了...
  400件処理完了...
  500件処理完了...
  600件処理完了...
  700件処理完了...
  800件処理完了...
  900件処理完了...
  1000件処理完了...

✅ 移行完了: 1000件成功, 0件エラー
📊 PostgreSQLのhorse_relationsテーブル: 1000件
